# Image Super Resolution using ESRGAN

---

## Technical information
- Research: Urban Safe Feminist design
- Creation date: 2024-05-22

## Objectives
Implement an upscaling algorithm that improves the resolution of a set of images obtained from the StreetView API.

## Summary
This notebook was developed to implement the upscaling algorithm. The framework used is TensorFlow in a Google Colab environment. The model used is ESRGAN which is available from https://www.tensorflow.org/hub/tutorials/image_enhancing.

## Results
- For each image of the original dataset, the ESRGAN model was applied to obtain its corresponding image in better quality.
- Metrics such as PSNR and SSIM are provided to evaluate the quality of the images generated at high resolution.

## Change log:
- 2024-05-22 Initial notebook creation



Algorithm selection: The ESRGAN algorithm preserves detail and visual quality through its ability to learn complex patterns. It uses a generative model, which helps to capture finer details in images, resulting in higher visual quality results. This also helps to contain realistic details, which is useful in a diverse number of applications and scenarios.

### Libraries

In [20]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import cv2
import random

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

%matplotlib inline
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

### Download the set of images

The images are contained in the `data` folder.  

In [2]:
!git clone https://github.com/DiversaStudio/Test.git

Cloning into 'Test'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (78/78), 19.59 MiB | 10.96 MiB/s, done.


### Declaring general variables

In [9]:
# Google libraries to make the connection with the drive
from google.colab import files
from google.colab import drive

drive.mount("/content/drive") # Request for connection to google drive account is made.

path_original_images_folder = r"/content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes"
path_sr_images_folder = r"/content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1" #Pretrained model ESRGAN

## Defining helper functions

### Define the function preprocess_image

The configuration for the application of the ESRGAN model has special features for its operation. At this stage the original images are preprocessed to meet the requirements of the model.

In [5]:
def preprocess_image(image_path):
    """ Loads image from path and preprocesses to make it model ready
        Args:
          image_path: Path to the image file
    """
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    # remove the alpha channel, the model only supports images with 3 color channels (RGB).
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4 #Adjusts image dimensions to be multiples of 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32) #Converts the image to the tf.float32 data type
    return tf.expand_dims(hr_image, 0)

### Define the function save_image

In [6]:
def save_image(image, filename, output_folder):
    """
      Saves unscaled Tensor Images.
      Args:
        image: 3D image tensor. [height, width, channels]
        filename: Name of the file to save.
        output_folder: Folder to save the image.
    """
    if not isinstance(image, Image.Image):
        image = tf.clip_by_value(image, 0, 255)
        image = Image.fromarray(tf.cast(image, tf.uint8).numpy()) #Converts the image tensor to an 8-bit unsigned integer array and then to a PIL image.
    print(f'image path: {os.path.join(output_folder, "%s.jpg" % filename)}')
    image.save(os.path.join(output_folder, "%s.jpg" % filename))
    print("Saved as %s.jpg" % filename)

## Evaluation of the model

### Calculate PSNR and SSIM

- These metrics were selected to complement the model evaluation. On the one hand, the PSNR helps to calculate how much noise exists in the processed image with respect to the original one respect of the quadratic error in term of decibels. According to preliminary research, values above 20 dB are optimal values for the PSRN result because it indicates that the output signal is very similar to the original signal.


In [21]:
def calculate_metrics(original_image, upscaled_image):
    """
    Calcula PSNR y SSIM entre la imagen original y la mejorada.

    Args:
        original_image (np.array): Imagen original.
        upscaled_image (np.array): Imagen mejorada.

    """
    # Convertir imágenes a arrays numpy
    original = np.array(original_image)
    upscaled = np.array(upscaled_image)

    # Redimensionar la imagen original al tamaño de la imagen mejorada
    original_resized = np.array(original_image.resize((upscaled.shape[1], upscaled.shape[0]), Image.BICUBIC))

    # Calcular PSNR y SSIM
    psnr_value = psnr(original_resized, upscaled)
    ssim_value = ssim(original_resized, upscaled, multichannel=True)

    return psnr_value, ssim_value


## Performing ESRGAN of images loaded from path

The original images will be selected and subjected to a preprocessing phase. This stage is necessary to fit the parameters of the model. Subsequently, the ESRGAN model will be applied to these preprocessed images to obtain the super resolution images that will be stored in the folder `imagenes_sr`.

In [23]:
# 1. Get list of images stored in the input folder
input_imagenes_list = os.listdir(path_original_images_folder)  # List of images

results = []

# Create folder if it does not exist
if not os.path.exists(path_sr_images_folder):
    os.makedirs(path_sr_images_folder)

# 2. Loading the model
model = hub.load(SAVED_MODEL_PATH)

# 3. Iterate over the list of images to apply the super resolution model and save each resulting image in the output folder.
for image in input_imagenes_list:
    # Open image
    image_path = os.path.join(path_original_images_folder, image)  # Define the path to each individual image
    base_name = os.path.basename(image_path)

    hr_image = preprocess_image(image_path)  # Open the image and apply the preprocess_image function

    fake_image = model(hr_image)  # Apply the model and obtain the resulting image.
    fake_image = tf.squeeze(fake_image).numpy()  # Convert tensor to numpy array

    result_image_name = image.split(".")[0] + "_sr"  # Name of the images

    save_image(fake_image, filename=result_image_name, output_folder=path_sr_images_folder)  # Save the final image

    # Calcular métricas
    original_image = Image.open(image_path).convert('RGB')
    fake_image_pil = Image.fromarray(np.uint8(fake_image * 255))  # Convert numpy array to PIL Image for metrics
    psnr_value, ssim_value = calculate_metrics(original_image, fake_image_pil)

    # Save results in the list
    results.append({'Imagen': base_name, 'PSNR': psnr_value, 'SSIM': ssim_value})

# Print the resulting list of images
print(os.listdir(path_sr_images_folder))

image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_140_0_sr.jpg
Saved as streetview_140_0_sr.jpg


<ipython-input-21-ba822c512820>:19: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(original_resized, upscaled, multichannel=True)


image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_139_270_sr.jpg
Saved as streetview_139_270_sr.jpg
image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_139_90_sr.jpg
Saved as streetview_139_90_sr.jpg
image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_139_180_sr.jpg
Saved as streetview_139_180_sr.jpg
image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_139_0_sr.jpg
Saved as streetview_139_0_sr.jpg
image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_138_90_sr.jpg
Saved as streetview_138_90_sr.jpg
image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_138_180_sr.jpg
Saved as streetview_138_180_sr.jpg
image path: /content/drive/MyDrive/Diversa/Urbansafe/data_extraction/imagenes_sr/streetview_140_270_sr.jpg
Saved as streetview_140_270_sr.jpg
image path: /c

In [24]:
# Create a DataFrame from the results list
pathMetricas = path_sr_images_folder+'/metricas_resultados_imagenes.csv'
results_df = pd.DataFrame(results)

# Saving the DataFrame in a CSV file
results_df.to_csv(pathMetricas, index=False)